In [2]:
import numpy as np
import os
import supervision as sv
import cv2
from PIL import Image
import supervision as sv
import groundingdino.datasets.transforms as T
import torch
from torchvision.ops import box_convert

In [3]:
import os
WEIGHTS_NAME = "groundingdino_swint_ogc.pth"
if os.path.isfile(WEIGHTS_NAME) is False:    
    !wget -q https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth
if os.path.isfile("GroundingDINO\groundingdino\config\GroundingDINO_SwinT_OGC.py") is False:
    !git clone https://github.com/IDEA-Research/GroundingDINO.git

In [4]:
os.path.isfile("GroundingDINO\groundingdino\config\GroundingDINO_SwinT_OGC.py")

True

In [5]:
from groundingdino.util.inference import load_model, load_image, predict, annotate
model = load_model("GroundingDINO\groundingdino\config\GroundingDINO_SwinT_OGC.py", "groundingdino_swint_ogc.pth")

c:\Users\marin\anaconda3\envs\Pytorch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\marin\anaconda3\envs\Pytorch\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


final text_encoder_type: bert-base-uncased


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [20]:
IMAGE_FOLDER = 'output'
LICENSE_PLATE_FOLDER = 'License Plate'
TEXT_PROMPT = "Number Plate"
BOX_THRESHOLD = 0.3
TEXT_THRESHOLD = 0.3
for file_name in os.listdir(IMAGE_FOLDER):
    if file_name.endswith('.jpg'):
        # load image
        image_path = os.path.join(IMAGE_FOLDER, file_name)
        image_source, image = load_image(image_path)
        # detect license plates
        boxes, logits, phrases = predict(
            model=model, 
            image=image, 
            caption=TEXT_PROMPT, 
            box_threshold=BOX_THRESHOLD, 
            text_threshold=TEXT_THRESHOLD
        )
        if len(boxes) ==0:
            continue
        # save license plate images to separate folder
        h, w, _ = image_source.shape
        boxes = boxes * torch.Tensor([w, h, w, h])
        xyxy = box_convert(boxes=boxes, in_fmt="cxcywh", out_fmt="xyxy").numpy()
        detections = sv.Detections(xyxy=xyxy)
        image_source=cv2.cvtColor(image_source, cv2.COLOR_BGR2RGB)
        for i in range(len(detections)):
            x1, y1, x2, y2 = detections.xyxy[i].astype(int)
            license_plate_file_name = f"{os.path.splitext(file_name)[0]}_{i}.jpg"
            license_plate_file_path = os.path.join(LICENSE_PLATE_FOLDER, license_plate_file_name)
            cv2.imwrite(license_plate_file_path,image_source[y1:y2,x1:x2])